# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will convert the json encoded request body into the image format. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the images and uses the first function to score them.

In [3]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model
from dotenv import set_key, get_key, find_dotenv
import logging
from testing_utilities import img_url_to_json

In [2]:
import keras
import tensorflow
print("Keras: ", keras.__version__)
print("Tensorflow: ", tensorflow.__version__)

Using TensorFlow backend.


Keras:  2.2.0
Tensorflow:  1.10.0


In [3]:
env_path = find_dotenv(raise_error_if_not_found=True)

Let's load the workspace.

In [4]:
_MODEL_NAME = 'resnet_model'

In [13]:
model_path

'model_resnet_weights.h5'

## Write and save driver script

In [8]:
import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

from azureml.core.model import Model

import numpy as np
import timeit as t
import base64
import json
from PIL import Image, ImageOps
from io import BytesIO

def _base64img_to_numpy(base64_img_string):
    decoded_img = base64.b64decode(base64_img_string)
    img_buffer = BytesIO(decoded_img)
    imageData = Image.open(img_buffer).convert("RGB")
    img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
    img = image.img_to_array(img)
    return img

def call_model(img_array, model, number_results):
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    preds = decode_predictions(preds, top=number_results)[0]       
    return preds





In [ ]:
model = ResNet152(weights='imagenet')

In [99]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"
jsonimg = img_url_to_json(IMAGEURL)
inputString = json.dumps(json.loads(jsonimg)['input'])

base64Dict = json.loads(inputString) 
for k, v in base64Dict.items():
    img_file_name, base64Img = k, v 
img_array = _base64img_to_numpy(base64Img)

preds = call_model(img_array, model,3)
print("check 1.....")
print(type(preds))
print(preds)
print("check 2 ....")
responses = {img_file_name: preds}

check 1.....
<class 'list'>
[('n02127052', 'lynx', 0.9816483), ('n02128385', 'leopard', 0.0077441484), ('n02123159', 'tiger_cat', 0.003686138)]
check 2 ....


In [20]:
type(preds[0][2])

numpy.float32

In [100]:
preds

[('n02127052', 'lynx', 0.9816483),
 ('n02128385', 'leopard', 0.0077441484),
 ('n02123159', 'tiger_cat', 0.003686138)]

In [77]:
def tuple_float_to_str(x):
    a= list(x)
    a[2]= str(a[2])
    b= tuple(a)
    return b

In [82]:
n=preds[0]
m = tuple_float_to_str(n)
print(type(m[1]))

<class 'str'>


In [101]:
a = preds
for i, s in enumerate(a):
    a[i] = tuple_float_to_str(s)
print(a)

[('n02127052', 'lynx', '0.9816483'), ('n02128385', 'leopard', '0.0077441484'), ('n02123159', 'tiger_cat', '0.003686138')]


In [98]:
type(a[0][2])

str

In [37]:
preds_str = list(map(str, [x[0:3] for x in preds]))

In [38]:
preds_str

["('n02127052', 'lynx', 0.9816483)",
 "('n02128385', 'leopard', 0.0077441484)",
 "('n02123159', 'tiger_cat', 0.003686138)"]

In [102]:
%%writefile driver.py

import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

from azureml.core.model import Model

import numpy as np
import timeit as t
import base64
import json
from PIL import Image, ImageOps
from io import BytesIO
import logging

number_results = 3
logger = logging.getLogger("model_driver")

def _base64img_to_numpy(base64_img_string):
    decoded_img = base64.b64decode(base64_img_string)
    img_buffer = BytesIO(decoded_img)
    imageData = Image.open(img_buffer).convert("RGB")
    img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
    img = image.img_to_array(img)
    return img

def create_scoring_func():
    """ Initialize ResNet 152 Model 
    """   
    start = t.default_timer()
    # model = ResNet152(weights='imagenet')
    model_name = 'resnet_model'
    model_path=Model.get_model_path(model_name)
    model = ResNet152()
    model.load_weights(model_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(img_array):
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        preds = model.predict(img_array)
        preds = decode_predictions(preds, top=number_results)[0] 
        return preds
    
    return call_model       


def tuple_float_to_str(x):
        """tuple x = ('n02127052', 'lynx', 0.9816483) convert the 3rd element type numpy.float32 to string
           return: ('n02127052', 'lynx', '0.9816483')
        """
        a= list(x)
        a[2]= str(a[2])
        b= tuple(a)
        return b
    
def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        base64Dict = json.loads(inputString) 
        for k, v in base64Dict.items():
            img_file_name, base64Img = k, v 
        img_array = _base64img_to_numpy(base64Img)
        preds = scoring_func(img_array)
        
        # convert the 3rd element type of preds items from numpy.float32 to string
        for i, s in enumerate(preds):
            preds[i] = tuple_float_to_str(s)

        responses = {img_file_name: preds}
        
        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return tf.__version__

def init():
    """ Initialise the model and scoring function
    """
    global process_and_score
    process_and_score = get_model_api()
    
def run(raw_data):
    """ Make a prediction based on the data passed in using the preloaded model
    """
    return process_and_score(json.dumps(json.loads(raw_data)['input']))

Overwriting driver.py


## Test the driver¶ 
We test the driver by passing data.

In [6]:
logging.basicConfig(level=logging.DEBUG)

In [7]:
%run driver.py

In [8]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

DEBUG:cli.azure.cli.core:Current cloud config:
AzureCloud
DEBUG:cli.azure.cli.core.util:attempting to read file /home/mylogin/.azure/accessTokens.json as utf-8-sig
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - Authority:Performing instance discovery: ...
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - Authority:Performing static instance discovery
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - Authority:Authority validated via static instance discovery
INFO:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - TokenRequest:Getting token from cache with refresh if necessary.
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:finding with query keys: {'_clientId': '...', 'userId': '...'}
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Looking for potential cache entries: {'_clientId': '...', 'userId': '...'}
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Found 2 potential entries.
DEBUG:adal-python:3e

Found the config file in: /data/home/mylogin/notebooks/yanzrepo/DistributedDeepLearning/AKSDeploymentTutorial_AML/Keras_Tensorflow/aml_config/config.json


DEBUG:urllib3.connectionpool:https://login.microsoftonline.com:443 "POST /72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token HTTP/1.1" 200 3304
INFO:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - OAuth2Client:Get Token Server returned this correlation_id: 3e1f8a29-74f5-43f5-babe-db772eee2262
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Created new cache entry from refresh response.
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Removing entry.
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Adding entry AccessTokenId: b'4++Ojr5UOlnb/W1BfAzN6wNgMlDh4UpGKDzSwPkXnqM=', RefreshTokenId: b'y85yfXPddjM3gdr367feA0PQgfQvBbm8b+sK2JQYpsI='
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Updating 1 cached refresh tokens
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Remove many: 1
DEBUG:adal-python:3e1f8a29-74f5-43f5-babe-db772eee2262 - CacheDriver:Add many: 1
INFO:adal-python:3e1f8a29-

yanzamlworkspace
yanzamlworkspace
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [9]:
model_path = Model.get_model_path(_MODEL_NAME, _workspace=ws)

DEBUG:azureml.core.run:Could not load run context Failed to load a submitted run, if outside of an execution context, use project.start_run to initialize an azureml.core.Run., switching offline: False
DEBUG:azureml.core.run:Could not load the run context and allow_offline set to False
DEBUG:azureml.core.model:RunEnvironmentException: Failed to load a submitted run, if outside of an execution context, use project.start_run to initialize an azureml.core.Run.
DEBUG:cli.azure.cli.core:Current cloud config:
AzureCloud
DEBUG:adal-python:04afbf0b-b2bc-40ca-935f-91115e07403b - Authority:Performing instance discovery: ...
DEBUG:adal-python:04afbf0b-b2bc-40ca-935f-91115e07403b - Authority:Performing static instance discovery
DEBUG:adal-python:04afbf0b-b2bc-40ca-935f-91115e07403b - Authority:Authority validated via static instance discovery
INFO:adal-python:04afbf0b-b2bc-40ca-935f-91115e07403b - TokenRequest:Getting token from cache with refresh if necessary.
DEBUG:adal-python:04afbf0b-b2bc-40ca-

DEBUG:azureml._restclient.clientbase.WorkerPool.CreateFutureFunc: _execute_with_base_arguments:[START]
DEBUG:msrest.service_client:Accept header absent and forced to application/json
DEBUG:azureml._restclient.clientbase.WorkerPool.CreateFutureFunc: _execute_with_base_arguments:[STOP]
DEBUG:msrest.universal_http.requests:Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
DEBUG:azureml.ArtifactsClient.list_sas_by_prefix:Using basic handler - no exception handling
DEBUG:azureml.ArtifactsClient.list_sas_by_prefix.WaitingTask:[START]
DEBUG:cli.azure.cli.core:Current cloud config:
AzureCloud
DEBUG:adal-python:bb575216-752c-4bf6-829c-ce186798c44e - Authority:Performing instance discovery: ...
DEBUG:adal-python:bb575216-752c-4bf6-829c-ce186798c44e - Authority:Performing static instance discovery
DEBUG:adal-python:bb575216-752c-4bf6-829c-ce186798c44e - Authority:Authority validated via static instance discovery
INFO:adal-python:bb575216-752c-4bf6-829c-ce186798c44e - TokenReque

In [10]:
IMAGEURL = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg"

In [ ]:
init()

DEBUG:azureml.core.run:Could not load run context Failed to load a submitted run, if outside of an execution context, use project.start_run to initialize an azureml.core.Run., switching offline: False
DEBUG:azureml.core.run:Could not load the run context and allow_offline set to False
DEBUG:azureml.core.model:RunEnvironmentException: Failed to load a submitted run, if outside of an execution context, use project.start_run to initialize an azureml.core.Run.
DEBUG:azureml.core.model:version is None. Latest version is 2
DEBUG:azureml.core.model:Found model path at azureml-models/resnet_model/2/model_resnet_weights.h5


In [ ]:
jsonimg = img_url_to_json(IMAGEURL)
resp = run(jsonimg)

In [4]:
jsonimg = img_url_to_json(IMAGEURL)

In [6]:
with open("jsonimg", "w") as f:
    f.write(jsonimg)